In [ ]:
import openai
import getpass
import pandas as pd
import re

In [ ]:
openai.api_key = getpass.getpass()

In [ ]:
test_df = pd.read_csv("topic_8_sample.csv")
test_df = test_df.head(10)
test_df

In [ ]:
systems_prompt = """
You are a Social Media Manager
"""

In [ ]:
# Assuming you have a dataset named 'your_dataset' with a column 'Content Description'
# Modify the main_prompt to include instructions for analyzing the dataset

main_prompt = f"""
Analyze the dataset provided below generate SEO Findings:


Dataset:
{test_df.to_string(index=False)}

---

# EDA (Exploratory Data Analysis)
Based on the analysis of the provided dataset, generate compelling titles, tags, and descriptions for Youtube content.
"""

# The rest of your code can remain similar to the previous examples.


In [ ]:
# Make sample dataframe
content_string = "I want to make a Youtube video about a baby girl gender reveal party"

In [ ]:
# Creating the while loop

total_score = 0

while total_score < 3:
    # Run the API
    reviews_string = content_string

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": systems_prompt
            },
            {
                "role": "user",
                "content": main_prompt + reviews_string
            }
        ]
    )

    content = response.choices[0].message['content'].strip()

    # Get the strings
    # Define regular expressions to extract title, tags, and description
    title_pattern = re.compile(r'Title: (.+)', re.IGNORECASE)
    tags_pattern = re.compile(r'Tags: (.+)', re.IGNORECASE)
    description_pattern = re.compile(r'Description: (.+)', re.IGNORECASE)

    # Extract information using regular expressions
    title_match = title_pattern.search(content)
    tags_match = tags_pattern.search(content)
    description_match = description_pattern.search(content)

    # Retrieve values from the matches or set default values if not found
    title = title_match.group(1) if title_match else ''
    tags = tags_match.group(1) if tags_match else ''
    description = description_match.group(1) if description_match else ''

    # Create separate strings for title, tags, and descriptions
    title_string = f"Title: {title}"
    tags_string = f"Tags: {tags}"
    descriptions_string = f"Description: {description}"

    # Get keys from the prompt
    stop_words = ['I', 'want', 'to', 'make', 'a', 'youtube', 'video', 'about']
    token_prompt = nltk.word_tokenize(content_string)
    key_prompt = [word for word in token_prompt if word.lower() not in stop_words]

    keywords = ['baby', 'reveal', 'gender', 'dream'] #from the list
    keywords.extend(key_prompt)

    # Scoring - TITLE Get score (0 or 1)
    # STEP 1: Tokenize the title
    key_title = nltk.word_tokenize(title_string)
    key_title = [word.lower() for word in key_title if word.lower() not in stop_words]

    # STEP 2: Compare tokens with keywords
    title_score = 1 if any(word in key_title for word in keywords) else 0

    # Scoring - TAGS Get score (0 or 1)
    # STEP 1: Tokenize the tags
    key_tag = nltk.word_tokenize(tags_string)
    key_tag = [word.lower() for word in key_tag if word.lower() not in stop_words]

    # STEP 2: Compare tokens with keywords
    tag_score = 1 if any(word in key_tag for word in keywords) else 0

    # Scoring - DESCRIPTION Get score (0 or 1)
    # STEP 1: Define Sentiment Analysis Function
    def rule_patterns_annotation(text):
        analysis = TextBlob(text)
        if analysis.sentiment.polarity > 0:
            return 'Positive'
        elif analysis.sentiment.polarity == 0:
            return 'Neutral'
        else:
            return 'Negative'

    # STEP 2: Apply the rule_patterns_annotation function to the video_description
    desc_sentiment = rule_patterns_annotation(descriptions_string)
    desc_score = 1 if desc_sentiment=="Positive" else 0

    total_score = title_score + tag_score + desc_score

# After the loop
if total_score >= 3:
    print(title_string)
    print(tags_string)
    print(descriptions_string)
else: print("Fail")